Credit: 
https://www.kaggle.com/nicohrubec/lgb-baseline-with-groupkfold
https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline

In [ ]:
#https://www.kaggle.com/valleyzw/ubiquant-lgbm-baseline
params = {
        'learning_rate':0.1,##学习率,每个树学习前一个树的残差步长
        "objective": "regression",##学习任务
        "metric": "rmse",##评价准则，均根方误差
        'boosting_type': "gbdt",##学习类型
        'verbosity': -1,##打印输出，大于0打印每棵树的进度和性能
        'n_jobs': -1, 
        'seed': 21,##随机种子-方便重现
        'lambda_l1': 1.1895057699067542, ##正则化
        'lambda_l2': 1.9079686837880768e-08, ##正则化
        'num_leaves': 112, ##叶结点最大样本数，防止过度拟合
        'subsample':0.8,##用于拟合个别基础学习器的样本分数 选择子样本<1.0导致方差的减少和偏差的增加
        'feature_fraction': 0.6259927292757151, ##设置在每次迭代中使用特征的比例
        'bagging_fraction': 0.9782210574588895, ##每次迭代时用的数据比例，用于加快训练速度和减小过拟合
        'bagging_freq': 1, ##防止过拟合
        'n_estimators': 1000, ##决策树的个数，弱学习器个数
        'max_depth': 12,##树的最大深度，控制树的大小，防止过度拟合        
        'max_bin': 494,##调节准确率 ，调小max_bin的值可以提高模型训练速度， 调大它的值和调大n_estimators效果一样
        'min_data_in_leaf': 366##每个叶子上的最少数据，调节训练速度，调大它的值可以防止过拟合，它的值通常设置的比较大
    }

In [ ]:
import numpy as np
import pandas as pd
import os, gc
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr as p
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

In [ ]:
##利用lightgbm库实现模型
def run(train):
    for fold, (trn_idx, val_idx) in enumerate(kfold.split(train[feats], train[target], groups=train['time_id'])):
        X_train, y_train = train.iloc[trn_idx][feats], train.iloc[trn_idx][target]
        X_val, y_val = train.iloc[val_idx][feats], train.iloc[val_idx][target]
        ##划分训练数据和测试数据
        model = lgb.LGBMRegressor(**params)
        ##模型训练
        model.fit(X_train, y_train, eval_metric='rmse', eval_set=[(X_train, y_train), (X_val, y_val)], verbose=100, early_stopping_rounds=50)
        ##模型预测
        y_pred = model.predict(X_val)

        score = p(y_pred, y_val)[0]
        print(f"Fold {fold + 1}: {score}")
        fold_scores.append(score)
        models.append(model)
        
        del model, y_pred, score
        gc.collect()
    
    print(f"Overall score: {np.mean(fold_scores)}")

In [ ]:
train = pd.read_pickle('../input/ump195gb/train.pkl')##读文件

In [ ]:
train['nun_time_ids'] = train['investment_id'].map(train.groupby('investment_id')['time_id'].nunique())
train.head(3)

In [ ]:
fold_scores = []
models = []
n_splits=5
feats = [f for f in train.columns if f not in ['time_id', 'row_id', 'target']]
target = 'target'
kfold = GroupKFold(n_splits=n_splits)

In [ ]:
run(train[train.time_id>1000])

In [ ]:
import ubiquant
env = ubiquant.make_env()  
iter_test = env.iter_test()

for i, (test_df, sample_prediction_df) in enumerate(iter_test):
    test_df['nun_time_ids'] = test_df['investment_id'].map(train.groupby('investment_id')['time_id'].nunique()) + (i+1) 
    
    test_df = test_df[feats]
    pred = np.zeros(len(test_df))
    
    for i in range(len(models)):
        pred += models[i].predict(test_df)
    
    pred /= len(models)
    sample_prediction_df['target'] = pred
    env.predict(sample_prediction_df)##模型评估